In [ ]:
# vad demo
import torch
from utils_vad import read_audio, get_speech_timestamps, init_jit_model,save_audio,collect_chunks

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = init_jit_model(model_path='./silero_vad.jit',device=torch.device(device))
wav = read_audio('./wav/BAC009S0150W0240.wav', sampling_rate=16000)
wav1 = wav.float()
wav1 = wav1.to(device)
# get speech timestamps from full audio file
speech_timestamps = get_speech_timestamps(wav1, model, sampling_rate=16000)
save_audio('BAC009S0150W0240.wav',
           collect_chunks(speech_timestamps, wav), sampling_rate=16000) 

In [ ]:
# vad 批量处理aishell语音
import os
import torch
import pandas as pd
from utils_vad import read_audio, get_speech_timestamps, init_jit_model,save_audio, collect_chunks

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = init_jit_model(model_path='./silero_vad.jit',device=torch.device(device))
save_path = '/mnt/'
df = pd.read_csv('../csv/wav1mic.csv')
df_s = pd.DataFrame(columns=['file_path'])
for i in range(0, len(df)):
    wav = read_audio(df.iloc[i]['file_path'], sampling_rate=16000)
    dir_path, file_name = os.path.split(df.iloc[i]['file_path'])
    print(file_name)
    wav1 = wav.float()
    wav1 = wav1.to(device)
    speech_timestamps = get_speech_timestamps(wav1, model, sampling_rate=16000)
    save_audio(save_path+file_name,
           collect_chunks(speech_timestamps, wav), sampling_rate=16000) 
    df_s = df_s.append({'file_path':save_path+file_name}, ignore_index=True)
df_s.to_csv('./vad1mic.csv',index=False,sep=',')

In [ ]:
# vad 处理实际数据,注意要分别对每个通道做
import os
import torch
import pandas as pd
import librosa
import soundfile as sf
from utils_vad import read_audio, get_speech_timestamps, init_jit_model,save_audio, collect_chunks

ls = []
for root, dirs, files in os.walk('./wav'):
    for f in files:
        ls.append(os.path.join(root, f))
       
print(ls)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = init_jit_model(model_path='./silero_vad.jit',device=torch.device(device))

save_path = './vad_wav/'
for i in range(0, len(ls)):
    #wav = read_audio(ls[i], sampling_rate=16000)
    dir_path, file_name = os.path.split(ls[i])
    wav, fs = sf.read(ls[i])
    wav = wav[:,:3].T
    wav = torch.from_numpy(wav)
    wav1 = wav.float()
    wav1 = wav1.to(device)
    speech_timestamps = get_speech_timestamps(wav1[i], model, sampling_rate=16000)
    wav_multichannel = []
    for j in range(0, wav1.shape[0]):
        wav_multichannel.append(collect_chunks(speech_timestamps, wav1[j])) 
    wav_multichannel = torch.stack(wav_multichannel)
    print(wav_multichannel.shape)
    sf.write(save_path+file_name, wav_multichannel.cpu().numpy().T, 16000)
#     save_audio(save_path+file_name,
#             wav_multichannel, sampling_rate=16000) 

In [ ]:
# vad 批量处理aishell语音
import os
import torch
import pandas as pd
from utils_vad import read_audio, get_speech_timestamps, init_jit_model,save_audio, collect_chunks

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = init_jit_model(model_path='./silero_vad.jit',device=torch.device(device))
save_path = '/mnt/users/daihuangyu/AEC_Challenge/AEC-Challenge/datasets/synthetic/'
  
if not os.path.exists(save_path + '/nearend_mic_signal_vad/'):
    os.makedirs(save_path + '/nearend_mic_signal_vad/') 
    os.makedirs(save_path + '/nearend_speech_vad/') 
    os.makedirs(save_path + '/echo_signal_vad/') 
    os.makedirs(save_path + '/farend_speech_vad/') 
    
for i in range(1407, 1408):
    wav = read_audio(save_path + 'nearend_mic_signal/' + f'nearend_mic_fileid_{i}.wav', sampling_rate=16000)
    print(f'nearend_mic_fileid_{i}.wav')
    wav_cuda = wav.float()
    wav_cuda = wav_cuda.to(device)
    speech_timestamps = get_speech_timestamps(wav_cuda, model, sampling_rate=16000)
    
    wav2 = read_audio(save_path + 'nearend_speech/' + f'nearend_speech_fileid_{i}.wav', sampling_rate=16000)
    wav3 = read_audio(save_path + 'echo_signal/' + f'echo_fileid_{i}.wav', sampling_rate=16000)
    wav4 = read_audio(save_path + 'farend_speech/' + f'farend_speech_fileid_{i}.wav', sampling_rate=16000)
    
    save_audio(save_path + 'nearend_mic_signal_vad/' + f'nearend_mic_fileid_{i}.wav',
           collect_chunks(speech_timestamps, wav), sampling_rate=16000) 
    save_audio(save_path + 'nearend_speech_vad/' + f'nearend_speech_fileid_{i}.wav',
           collect_chunks(speech_timestamps, wav2), sampling_rate=16000)
    save_audio(save_path + 'echo_signal_vad/' + f'echo_fileid_{i}.wav',
           collect_chunks(speech_timestamps, wav3), sampling_rate=16000)
    save_audio(save_path + 'farend_speech_vad/' + f'farend_speech_fileid_{i}.wav',
           collect_chunks(speech_timestamps, wav4), sampling_rate=16000) 
    

